In [ ]:
# | include: false
# | default_exp scilint

In [ ]:
# | export

import ast
import json
import operator
import os
import re
import shutil
import sys
import warnings
from collections import Counter
from configparser import InterpolationMissingOptionError
from pathlib import Path
from typing import Any, Callable, Dict, Iterable, Tuple

import nbformat
import numpy as np
import pandas as pd
import yaml
from execnb.nbio import read_nb
from fastcore.script import call_parse
from nbdev.clean import nbdev_clean
from nbdev.config import get_config
from nbdev.doclinks import nbdev_export, nbglob
from nbdev.quarto import nbdev_docs, nbdev_readme
from nbdev.test import nbdev_test
from nbqa.__main__ import _get_configs, _main
from nbqa.cmdline import CLIArgs
from nbqa.find_root import find_project_root

In [ ]:
%load_ext autoreload
%autoreload 2

# Test Data Prep

In [ ]:
nbdev_path = Path(Path(".").resolve(), "example_nbs", "nbdev.ipynb")
nbdev_hq_path = Path(Path(".").resolve(), "example_nbs", "nbdev_high_quality.ipynb")
non_nbdev_path = Path(Path(".").resolve(), "example_nbs", "non_nbdev.ipynb")
non_nbdev_lq_path = Path(
    Path(".").resolve(), "example_nbs", "non_nbdev_low_quality.ipynb"
)
index_path = Path(Path(".").resolve(), "index.ipynb")
syntax_error_path = Path(Path(".").resolve(), "syntax_error.ipynb")

nbdev_nb = read_nb(nbdev_path)
nbdev_hq_nb = read_nb(nbdev_hq_path)
non_nbdev_nb = read_nb(non_nbdev_path)
non_nbdev_lq_nb = read_nb(non_nbdev_lq_path)
index = read_nb(index_path)
syntax_error = read_nb(index_path)

# `scilint_tidy` 

simple wrapper around no-decisions version of nbqa

In [ ]:
# | export


def run_nbqa_cmd(cmd):
    print(f"Running {cmd}")
    project_root: Path = find_project_root(tuple([str(Path(".").resolve())]))
    args = CLIArgs.parse_args([cmd, str(project_root)])
    configs = _get_configs(args, project_root)
    output_code = _main(args, configs)
    return output_code

In [ ]:
project_root: Path = find_project_root(tuple([str(Path(".").resolve())]))
assert os.path.basename(project_root) == "scilint"

In [ ]:
# | export


def tidy():
    tidy_tools = ["black", "isort", "autoflake"]
    [run_nbqa_cmd(c) for c in tidy_tools]

# Helpers

In [ ]:
# | export


def is_nbdev_project(project_path: Path = Path(".")):
    is_nbdev = True
    project_root = find_project_root(tuple([str(project_path.resolve())]))

    if not Path(project_root, "settings.ini").exists():
        is_nbdev = False
    try:
        get_config().lib_name
    except InterpolationMissingOptionError:
        is_nbdev = False

    return is_nbdev

In [ ]:
assert is_nbdev_project()

In [ ]:
import tempfile

with tempfile.TemporaryDirectory() as tmp_dir:
    assert not is_nbdev_project(Path(tmp_dir))

In [ ]:
# | export


def get_func_defs(code, ignore_private_prefix=True):
    func_names = []
    for stmt in ast.walk(ast.parse(code)):
        if isinstance(stmt, ast.FunctionDef):
            inner_cond = (
                False if ignore_private_prefix and stmt.name.startswith("_") else True
            )
            if inner_cond:
                func_names.append(stmt.name)
    return func_names

In [ ]:
test_code = """
x()
def y():
    pass
def z():
    def a():
        pass
class A():
    def b():
        pass
def blabla():
    return 1
def _hidden():
    return None
"""
func_defs = ["a", "b", "blabla", "y", "z"]
assert func_defs == sorted(get_func_defs(test_code))

In [ ]:
# | export


def count_func_calls(code, func_defs):
    func_calls = Counter({k: 0 for k in func_defs})
    for stmt in ast.walk(ast.parse(code)):
        if isinstance(stmt, ast.Call):
            if type(stmt.func) == ast.Subscript:
                func_name = stmt.func.value.id
            else:
                func_name = (
                    stmt.func.id if "id" in stmt.func.__dict__ else stmt.func.attr
                )
            if func_name in func_defs:
                if func_name in func_calls:
                    func_calls[func_name] += 1
    return func_calls

In [ ]:
test_code = """self.hierarchical_topic_reduction(3); 
topic_reduction(3); 
lambda x: topic(x); 
hierarchical_topic_reduction[4]; 
hierarchical_topic_reduction(4); 
blabla()
lambda y: other(5)
funcs = [x, y]
funcs[0](3)
"""
test_func_defs = [
    "topic",
    "topic_reduction",
    "blablabla",
    "hierarchical_topic_reduction",
]

In [ ]:
assert count_func_calls(test_code, test_func_defs) == Counter(
    {
        "topic": 1,
        "topic_reduction": 1,
        "blablabla": 0,
        "hierarchical_topic_reduction": 2,
    }
)

In [ ]:
nb_cell_code = r"""
def something():
    pass; pass # in x 2
    
%load_ext autoreload
%autoreload 2

!ls -l
if 1!= 2:
    print(4)
#| export

import pandas as pd # out
from sciflow.utils import lib_path, odbc_connect, query # out

#| export

def nb_to_sagemaker_pipeline(
    nb_path: Path,
    silent: bool = True,
):
    nb = read_nb(nb_path)  # in
    lib_name = get_config().get("lib_name")  # in
    module_name = find_default_export(nb["cells"])  # in
    
x = [1,2,3] # out
nb_to_sagemaker_pipeline() # out
"""

In [ ]:
# | export


def replace_ipython_magics(code):
    # Replace Ipython magic and shell command symbol with comment
    code = code.replace("%", "#")
    code = re.sub(r"^!", "#", code)
    return re.sub(r"\n\W?!", "\n#", code)

In [ ]:
throws = False
try:
    assert ast.parse(nb_cell_code)
except SyntaxError:
    throws = True
assert throws
assert type(ast.parse(replace_ipython_magics(nb_cell_code))) == ast.Module

In [ ]:
# | export


def safe_div(numer, denom):
    return 0 if denom == 0 else numer / denom

In [ ]:
assert safe_div(1, 1) == 1
assert safe_div(2, 1) == 2
assert safe_div(1, 2) == 0.5
assert safe_div(0, 1) == 0
assert safe_div(1, 0) == 0
assert safe_div(10, 1) == 10

In [ ]:
# | export


def get_cell_code(nb):
    pnb = nbformat.from_dict(nb)
    nb_cell_code = "\n".join(
        [
            replace_ipython_magics(c["source"])
            for c in pnb.cells
            if c["cell_type"] == "code"
        ]
    )
    return nb_cell_code

# Potential Quality Indicators

## 1. Calls-per-Function

In [ ]:
# | export


def calls_per_func(nb):
    nb_cell_code = get_cell_code(nb)
    func_defs = get_func_defs(nb_cell_code)
    func_calls = count_func_calls(nb_cell_code, func_defs)
    return func_calls

In [ ]:
# | export


def mean_cpf(nb):
    return pd.Series(calls_per_func(nb)).mean()

In [ ]:
# | export


def median_cpf(nb):
    with warnings.catch_warnings():
        warnings.filterwarnings(action="ignore", message="Mean of empty slice")
        return pd.Series(calls_per_func(nb)).median()

In [ ]:
assert mean_cpf(nbdev_nb).round(2) == 2.23
assert median_cpf(nbdev_nb) == 1

In [ ]:
assert mean_cpf(read_nb(nbdev_path)).round(2) == 2.23
assert mean_cpf(read_nb(nbdev_hq_path)).round(2) == 2.5
assert mean_cpf(read_nb(non_nbdev_path)).round(2) == 1.0
assert mean_cpf(read_nb(non_nbdev_lq_path)).round(2) == 1.62
assert pd.isnull(mean_cpf(index))

In [ ]:
assert median_cpf(read_nb(nbdev_path)) == 1.0
assert median_cpf(read_nb(nbdev_hq_path)).round(2) == 1.5
assert median_cpf(read_nb(non_nbdev_path)).round(2) == 1.0
assert median_cpf(read_nb(non_nbdev_lq_path)).round(2) == 1.0
assert pd.isnull(median_cpf(index))

## 2. Asserts-to-Function Ratio

In [ ]:
asserted_code = r"""
def something():
    pass; pass # in x 2
    
assert True

#| export

def nb_to_sagemaker_pipeline(
    nb_path: Path,
    silent: bool = True,
):
    nb = read_nb(nb_path)  # in
    lib_name = get_config().get("lib_name")  # in
    module_name = find_default_export(nb["cells"])  # in
    
x = [1,2,3] # out
assert len(x) > 2
assert something() is None # something +1

def tr():
    return True
    
def get_seg(num):
    return 2
    
assert(tr)
assert(tr()) # tr +1
assert(tr() == 4) # tr +1
assert(4 ==tr()) # tr +1
assert 0 != 0
assert "' '".join(tr(1)) == "00" # tr +1
assert len(get_seg(50)) == 50 # get_seg +1
assert max([int(x) for x in get_seg(100)]) == 99 # get_seg +1
"""

In [ ]:
import nbformat as nbf

In [ ]:
asserted_nb = nbf.v4.new_notebook()
asserted_nb["cells"] = [nbf.v4.new_code_cell(asserted_code)]

In [ ]:
# | export


def afr(nb):
    nb_cell_code = get_cell_code(nb)
    if nb_cell_code == "":  # no code cells - metric is not well defined
        return np.nan
    func_defs = get_func_defs(nb_cell_code)
    num_funcs = len(func_defs)

    assert_count = 0
    for stmt in ast.walk(ast.parse(nb_cell_code)):
        if isinstance(stmt, ast.Assert):
            assert_count += 1

    return safe_div(assert_count, num_funcs)

In [ ]:
assert afr(nbdev_nb) > 1
assert afr(nbdev_hq_nb) > 1
assert afr(non_nbdev_nb) == 0
assert afr(non_nbdev_lq_nb) == 0
assert pd.isnull(afr(index))

## 3. In-line Asserts Per Function

In [ ]:
# | export


def count_inline_asserts(code, func_defs):
    inline_func_asserts = Counter({k: 0 for k in func_defs})

    for stmt in ast.walk(ast.parse(code)):
        if isinstance(stmt, ast.Assert):
            for assert_st in ast.walk(stmt):
                if isinstance(assert_st, ast.Call):
                    func_name = (
                        assert_st.func.id
                        if "id" in assert_st.func.__dict__
                        else assert_st.func.attr
                    )
                    if func_name in inline_func_asserts:
                        inline_func_asserts[func_name] += 1
    return inline_func_asserts

In [ ]:
# | export


def iaf(nb):
    nb_cell_code = get_cell_code(nb)
    if nb_cell_code == "":
        return np.nan
    func_defs = get_func_defs(nb_cell_code)
    return count_inline_asserts(nb_cell_code, func_defs)

In [ ]:
func_defs = get_func_defs(asserted_code)
inline_asserts_expected = Counter(
    {"something": 1, "tr": 4, "get_seg": 2, "nb_to_sagemaker_pipeline": 0}
)
inline_asserts_actual = count_inline_asserts(asserted_code, func_defs)

In [ ]:
assert inline_asserts_actual == inline_asserts_expected

In [ ]:
assert 0.0 == pd.Series(iaf(nbdev_nb)).median()
assert 0.0 == pd.Series(iaf(nbdev_hq_nb)).median()
assert 0.0 == pd.Series(iaf(non_nbdev_nb)).median()
assert 0.0 == pd.Series(iaf(non_nbdev_lq_nb)).median()
with warnings.catch_warnings():
    warnings.filterwarnings(action="ignore", message="Mean of empty slice")
    assert pd.isnull(pd.Series(iaf(index)).median())

In [ ]:
iaf(non_nbdev_nb)

Counter({'scalar': 0, 'py_advanced': 0, 'pandas': 0})

In [ ]:
iaf(non_nbdev_lq_nb)

Counter({'get_traffic_text': 0,
         'get_experiment_segment': 0,
         'evaluate': 0,
         'serve_num_topics': 0,
         'get_num_topics': 0,
         'get_topic_sizes': 0,
         'get_topics': 0,
         'plot_wordcloud': 0})

In [ ]:
assert inline_asserts_expected == iaf(asserted_nb)

In [ ]:
# | export


def mean_iaf(nb):
    return pd.Series(iaf(nb)).mean()

In [ ]:
# | export


def median_iaf(nb):
    with warnings.catch_warnings():
        warnings.filterwarnings(action="ignore", message="Mean of empty slice")
        return pd.Series(iaf(nb)).median()

* What about calculating functions code coverage
* How does pytest-cov do it?

## 4. In-function Percentage

In [ ]:
# | export


def calc_ifp(nb_cell_code):
    stmts_in_func = 0
    stmts_outside_func = 0
    for stmt in ast.walk(ast.parse(replace_ipython_magics(nb_cell_code))):
        if isinstance(stmt, ast.FunctionDef) and not stmt.name.startswith("_"):
            for body_item in stmt.body:
                stmts_in_func += 1
        elif isinstance(stmt, ast.Module):
            for body_item in stmt.body:
                if not isinstance(body_item, ast.FunctionDef):
                    stmts_outside_func += 1
    return (
        0
        if stmts_outside_func + stmts_in_func == 0
        else (stmts_in_func / (stmts_outside_func + stmts_in_func)) * 100
    )

In [ ]:
assert (calc_ifp(nb_cell_code)) == (5 / (5 + 5)) * 100

In [ ]:
# | export


def ifp(nb):
    nb_cell_code = "\n".join(
        [
            replace_ipython_magics(c["source"])
            for c in nb.cells
            if c["cell_type"] == "code"
        ]
    )
    if nb_cell_code == "":
        return np.nan
    return calc_ifp(nb_cell_code)

In [ ]:
assert ifp(nbdev_nb) >= 0
assert ifp(nbdev_hq_nb) >= 0
assert ifp(non_nbdev_nb) >= 0
assert ifp(non_nbdev_lq_nb) >= 0
assert pd.isnull(ifp(index))

## 5. Markdown to Code Percent

In [ ]:
# | export


def mcp(nb):
    md_cells = [c for c in nb.cells if c["cell_type"] == "markdown"]
    code_cells = [c for c in nb.cells if c["cell_type"] == "code"]
    num_code_cells = len(code_cells)
    if num_code_cells == 0:
        return np.nan
    num_md_cells = len(md_cells)
    return (
        100
        if num_code_cells == 0
        else (num_md_cells / (num_md_cells + num_code_cells)) * 100
    )

In [ ]:
assert mcp(nbdev_nb) >= 0
assert mcp(nbdev_hq_nb) >= 0
assert mcp(non_nbdev_nb) >= 0
assert mcp(non_nbdev_lq_nb) >= 0
assert pd.isnull(mcp(index))

## 6. Total Code Length

In [ ]:
# | export


def tcl(nb):
    return sum([len(c["source"]) for c in nb.cells if c["cell_type"] == "code"])

In [ ]:
assert tcl(nbdev_nb) >= 50
assert tcl(nbdev_hq_nb) >= 50
assert tcl(non_nbdev_nb) >= 50
assert tcl(non_nbdev_lq_nb) >= 50
assert tcl(index) == 0

## 7. Lines-of-code per Markdown Section

In [ ]:
# | export


def loc_per_md_section(nb):
    num_md_sections = len(
        [
            c["source"]
            for c in nb.cells
            if c["cell_type"] == "markdown" and c["source"].strip().startswith("#")
        ]
    )
    total_code_len = tcl(nb)
    if total_code_len == 0 or num_md_sections == 0:
        result = np.nan
    else:
        result = tcl(nb) / num_md_sections
    return result

In [ ]:
assert loc_per_md_section(nbdev_nb) < 1000
assert loc_per_md_section(nbdev_hq_nb) < 1000
assert loc_per_md_section(non_nbdev_nb) is np.nan
assert loc_per_md_section(non_nbdev_lq_nb) > 1000
assert loc_per_md_section(index) is np.nan

## Quality Indicator Function Map

> Add new quality indicators here to be used. Signature contract is nb -> number. TODO: provide a proper typed signature, handle bools.

In [ ]:
# | export

indicator_funcs = {
    "calls_per_func_mean": mean_cpf,
    "calls_per_func_median": median_cpf,
    "asserts_func_ratio": afr,
    "inline_asserts_per_func_mean": mean_iaf,
    "inline_asserts_per_func_median": median_iaf,
    "in_func_pct": ifp,
    "markdown_code_pct": mcp,
    "loc_per_md_section": loc_per_md_section,
    "total_code_len": tcl,
}

# Linting Functions

## `lint_nb`

In [ ]:
# | export


def lint_nb(
    nb_path: Path,
    conf: Dict[str, Any],
    indicators: Dict[str, Callable],
    include_in_scoring: bool,
) -> Tuple[float]:
    nb = read_nb(nb_path)

    has_syntax_error = False
    indic_vals = list(np.repeat(np.nan, len(indicators)))
    try:
        for i, indic_name in enumerate(indicators):
            indic_vals[i] = round(indicators[indic_name](nb), conf["precision"])
    except SyntaxError as se:
        if conf["print_syntax_errors"]:
            print(f"Syntax error in notebook: {nb_path} reason: ", se)
        has_syntax_error = True
    indic_vals.append(has_syntax_error)
    indic_vals.append(include_in_scoring)

    return tuple(indic_vals)

In [ ]:
# | export


def get_excluded_paths(paths: Iterable[Path], exclude_pattern: str) -> Iterable[Path]:
    excl_paths = []
    for ex_pattern in exclude_pattern.split(","):
        ex_path = Path(ex_pattern)
        if ex_path.exists():
            excl_paths.extend([p for p in paths if ex_pattern in str(p)])
        elif not ex_path.exists():
            raise ValueError(f"Path component: {ex_path} does not exist")
        else:
            raise ValueError(
                f"Invalid exclusion pattern: {ex_path} pattern is comma separrated list of 'dir/' for directories and 'name.ipynb' for specific notebook"
            )
    return excl_paths

In [ ]:
paths = [Path(p) for p in nbglob(Path("."))]
assert sorted(
    [
        p.name
        for p in get_excluded_paths(paths, exclude_pattern="example_nbs/,index.ipynb")
    ]
) == sorted(
    [
        "non_nbdev_low_quality.ipynb",
        "nbdev_high_quality.ipynb",
        "non_nbdev.ipynb",
        "nbdev.ipynb",
        "index.ipynb",
        "syntax_error.ipynb",
    ]
)
assert sorted(
    [
        p.name
        for p in get_excluded_paths(
            paths, exclude_pattern="example_nbs/nbdev.ipynb,index.ipynb"
        )
    ]
) == sorted(["nbdev.ipynb", "index.ipynb"])

## `lint_nbs`

In [ ]:
# | export


def lint_nbs(conf: Dict[str, Any], indicators: Dict[str, Callable]):
    nb_paths = [Path(p) for p in nbglob(Path("."))]

    excluded_paths = None
    exclusions = conf["exclusions"]
    if exclusions is not None:
        excluded_paths = get_excluded_paths(nb_paths, exclude_pattern=exclusions)

    results = []
    nb_names = []
    for nb_path in nb_paths:
        include_in_scoring = True
        if exclusions is not None:
            include_in_scoring = False if nb_path in excluded_paths else True

        nb_names.append(nb_path.stem)
        lint_result = lint_nb(nb_path, conf, indicators, include_in_scoring)
        results.append(lint_result)

    lint_report = pd.DataFrame.from_records(
        data=results,
        index=nb_names,
        columns=list(indicators.keys()) + ["has_syntax_error", "include_in_scoring"],
    ).sort_values(["in_func_pct", "markdown_code_pct"], ascending=False)

    scoring_report = lint_report[lint_report.include_in_scoring].copy()
    all_warns, num_warnings = _calculate_warnings(scoring_report, conf)
    _persist_results(lint_report, all_warns, conf)

    if num_warnings > 0:
        display_warning_report(all_warns)
    else:
        print("No issues found during linting")

    return lint_report, all_warns, num_warnings

## `display_warning_report`

In [ ]:
# | export


def display_warning_report(all_warns: pd.DataFrame):
    print(
        "\n*********************************Begin Scilint Warning Report********************************"
    )
    print(all_warns.to_markdown(tablefmt="grid", index=False))
    print(
        "\n*********************************End Scilint Warning Report**********************************\n"
    )

## Internal Functions

### `_persist_results`

In [ ]:
# | export


def _persist_results(
    lint_report: pd.DataFrame, all_warns: pd.DataFrame, conf: Dict[str, Any]
):
    out_dir = Path(conf["out_dir"])
    conf_to_persist = {k: v for k, v in conf.items() if k != "indicators"}
    if not out_dir.exists():
        Path(out_dir).mkdir()
    with open(Path(out_dir, "scilint_config.json"), "w") as outfile:
        json.dump(conf_to_persist, outfile)
    all_warns.to_csv(Path(out_dir, "scilint_warnings.csv"), index=False)
    lint_report.to_csv(Path(out_dir, "scilint_report.csv"))

In [ ]:
with tempfile.TemporaryDirectory() as tmp_dir:
    report = pd.DataFrame({"a": [1, 2, 3]})
    _persist_results(report, report, {"indicators": [], "out_dir": tmp_dir})
    assert pd.read_csv(Path(tmp_dir, "scilint_report.csv"), index_col=0).equals(
        pd.DataFrame({"a": [1, 2, 3]})
    )
    assert pd.read_csv(Path(tmp_dir, "scilint_warnings.csv")).equals(
        pd.DataFrame({"a": [1, 2, 3]})
    )
    with open(Path(tmp_dir, "scilint_config.json")) as infile:
        assert json.load(infile) == {"out_dir": tmp_dir}

### `_calculate_warnings`

In [ ]:
# | export


def _calculate_warnings(
    scoring_report: pd.DataFrame, conf: Dict[str, Any], include_missing: bool = False
) -> Tuple[Dict[str, Any], int]:
    warning_details = []
    for op_text in list(conf["warnings"].keys()):
        for indic in conf["warnings"][op_text]:
            metric_series = scoring_report[indic]
            or_exp = pd.isnull(metric_series) if include_missing else False
            op = (
                operator.lt
                if op_text == "lt"
                else operator.gt
                if op_text == "gt"
                else operator.eq
            )
            warning_data = metric_series[
                (op(metric_series, conf["warnings"][op_text][indic])) | (or_exp)
            ]
            warning_dict = warning_data.to_dict()
            for key, val in warning_dict.items():
                warning_dict[key] = (
                    indic,
                    val,
                    op_text,
                    conf["warnings"][op_text][indic],
                )
            warning_details.append(warning_dict)

    all_warns = _reshape_warnings(scoring_report, warning_details)
    num_warnings = len(all_warns)
    return all_warns, num_warnings

### `_reshape_warnings`

In [ ]:
# | export


def _reshape_warnings(
    scoring_report: pd.DataFrame, warning_details: Iterable[Any]
) -> Dict[str, Iterable[Tuple]]:
    warnings_by_nb = {nb: [] for nb in scoring_report.index}
    for nb in scoring_report.index:
        for wd in warning_details:
            if nb in wd:
                warnings_by_nb[nb].append(tuple([nb] + list(wd[nb])))
    warnings_by_nb = {key: val for key, val in warnings_by_nb.items() if len(val) > 0}
    flattened_warns = [item for sublist in warnings_by_nb.values() for item in sublist]
    return pd.DataFrame.from_records(
        data=flattened_warns,
        columns=["notebook", "indicator", "value", "operator", "threshold"],
    )

In [ ]:
conf = yaml.safe_load(Path("scilint-default.yaml").read_text())

In [ ]:
lint_report, all_warns, num_warns = lint_nbs(conf, indicator_funcs)
assert num_warns == 7

************Begin Scilint Warning Report************
+----+-----------------------+------------------------------+---------+------------+-------------+
|    | notebook              | indicator                    |   value | operator   |   threshold |
+====+=======================+==============================+=========+============+=============+
|  0 | non_nbdev_low_quality | asserts_func_ratio           |       0 | lt         |         1   |
+----+-----------------------+------------------------------+---------+------------+-------------+
|  1 | non_nbdev_low_quality | inline_asserts_per_func_mean |       0 | lt         |         0.5 |
+----+-----------------------+------------------------------+---------+------------+-------------+
|  2 | non_nbdev_low_quality | loc_per_md_section           |    2955 | gt         |      1000   |
+----+-----------------------+------------------------------+---------+------------+-------------+
|  3 | non_nbdev             | asserts_func_ratio       

In [ ]:
conf["exclusions"] = """example_nbs/,index.ipynb"""
_, all_warns, num_warns = lint_nbs(conf, indicator_funcs)
assert num_warns == 0
conf["exclusions"] = None

In [ ]:
conf[
    "exclusions"
] = """example_nbs/non_nbdev_low_quality.ipynb,index.ipynb,example_nbs/syntax_error.ipynb"""
_, all_warns, num_warns = lint_nbs(conf, indicator_funcs)
assert num_warns == 3
conf["exclusions"] = None

************Begin Scilint Warning Report************
+----+------------+------------------------------+---------+------------+-------------+
|    | notebook   | indicator                    |   value | operator   |   threshold |
+====+============+==============================+=========+============+=============+
|  0 | non_nbdev  | asserts_func_ratio           |       0 | lt         |         1   |
+----+------------+------------------------------+---------+------------+-------------+
|  1 | non_nbdev  | inline_asserts_per_func_mean |       0 | lt         |         0.5 |
+----+------------+------------------------------+---------+------------+-------------+
|  2 | non_nbdev  | markdown_code_pct            |       0 | lt         |         5   |
+----+------------+------------------------------+---------+------------+-------------+
************End Scilint Warning Report************


In [ ]:
conf["exclusions"] = """example_nbs/non_nbdev_low_quality.ipynb"""
_, all_warns, num_warns = lint_nbs(conf, indicator_funcs)
assert num_warns == 4
conf["exclusions"] = None

************Begin Scilint Warning Report************
+----+--------------+------------------------------+---------+------------+-------------+
|    | notebook     | indicator                    |   value | operator   |   threshold |
+====+==============+==============================+=========+============+=============+
|  0 | non_nbdev    | asserts_func_ratio           |       0 | lt         |         1   |
+----+--------------+------------------------------+---------+------------+-------------+
|  1 | non_nbdev    | inline_asserts_per_func_mean |       0 | lt         |         0.5 |
+----+--------------+------------------------------+---------+------------+-------------+
|  2 | non_nbdev    | markdown_code_pct            |       0 | lt         |         5   |
+----+--------------+------------------------------+---------+------------+-------------+
|  3 | syntax_error | has_syntax_error             |       1 | equals     |         1   |
+----+--------------+--------------------------

### `_lint`

In [ ]:
# | export


def _lint(conf: Dict[str, Any]):
    _, _, num_warnings = lint_nbs(conf, indicator_funcs)
    fail_over = conf["fail_over"]
    if fail_over == -1:
        print("Linting outcome ignored as fail_over set to -1")
    elif num_warnings > fail_over:
        print(
            f"Linting failed: total warnings ({num_warnings}) exceeded threshold ({fail_over})"
        )
        sys.exit(num_warnings)
    else:
        print("Linting succeeded")

### `_build`

In [ ]:
# | export


def _build(conf: Dict[str, Any]):
    print("Tidying notebooks..")
    tidy()
    if is_nbdev_project():
        nbdev_export.__wrapped__()
        print("Converted notebooks to modules")
        print("Testing notebooks..")
        nbdev_test.__wrapped__()
    print("Running notebook linter..")
    _lint(conf)
    if is_nbdev_project():
        nbdev_clean.__wrapped__()
        print("Cleaned notebooks")

### `_load_conf`

In [ ]:
# | export


def _load_conf(
    conf_path: str = None,
    exclusions: str = None,
    fail_over: int = None,
    out_dir: int = None,
    precision: int = None,
    print_syntax_errors: bool = None,
):
    if conf_path is None:
        project_root = find_project_root(tuple([str(Path(".").resolve())]))
        conf_path = Path(project_root, "nbs", "scilint-default.yaml")
        print(f"Loading default lint config: {conf_path}")
    else:
        conf_path = Path(conf_path)

    conf = yaml.safe_load(conf_path.read_text())
    override_names = (
        "exclusions",
        "fail_over",
        "out_dir",
        "precision",
        "print_syntax_errors",
    )
    overrides = (exclusions, fail_over, out_dir, precision, print_syntax_errors)
    for override in zip(override_names, overrides):
        if override[1] is not None:
            conf[override[0]] = override[1]
    return conf

# Console Scripts

## `scilint_tidy`

In [ ]:
# | export


@call_parse
def scilint_tidy():
    tidy()

## `scilint_lint`

In [ ]:
# | export


@call_parse
def scilint_lint(
    conf_path: str = None,
    exclusions: str = None,
    fail_over: int = None,
    out_dir: int = None,
    precision: int = None,
    print_syntax_errors: bool = None,
):
    conf = _load_conf(
        conf_path, exclusions, fail_over, out_dir, precision, print_syntax_errors
    )
    _lint(conf)

## `scilint_build`

In [ ]:
# | export


@call_parse
def scilint_build(
    conf_path: str = None,
    exclusions: str = None,
    fail_over: int = None,
    out_dir: int = None,
    precision: int = None,
    print_syntax_errors: bool = None,
):
    conf = _load_conf(
        conf_path, exclusions, fail_over, out_dir, precision, print_syntax_errors
    )
    _build(conf)

## `scilint_ci`

In [ ]:
# | export


@call_parse
def scilint_ci(
    conf_path: str = None,
    exclusions: str = None,
    fail_over: int = None,
    out_dir: int = None,
    precision: int = None,
    print_syntax_errors: bool = None,
):
    if not is_nbdev_project():
        print("scilint_ci feature is only available for nbdev projects")
        return

    conf = _load_conf(
        conf_path, exclusions, fail_over, out_dir, precision, print_syntax_errors
    )

    _build(conf)

    if not shutil.which("quarto"):
        print(
            "Quarto is not installed. A working quarto install is required for the CI build"
        )
        sys.exit(-1)
    nbdev_readme.__wrapped__()
    nbdev_docs.__wrapped__()